In [2]:
!pip install tensorflow

    100% |████████████████████████████████| 45.3MB 23kB/s  eta 0:00:01    84% |███████████████████████████▏    | 38.4MB 17.4MB/s eta 0:00:01
    100% |████████████████████████████████| 3.1MB 411kB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 7.6MB/s eta 0:00:01
    100% |████████████████████████████████| 1.5MB 824kB/s eta 0:00:01
    100% |████████████████████████████████| 1.2MB 1.0MB/s eta 0:00:01
    100% |████████████████████████████████| 890kB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 7.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /Users/rishi/Library/Caches/pip/wheels/76/f7/0c/88796d7212af59bb2f496b12267e0605f205170781e9b86479
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /Users/rishi/Library/Caches/pip/wheels/8e/fa/d6/77dd17d18ea23fd7b860e02623d27c1be451521af40dd4a13e
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /Users/rishi/Librar

In [4]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 606kB/s ta 0:00:01    33% |██████████▊                     | 471kB 970kB/s eta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import tensor_array_ops
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.7.0


In [2]:
import pickle
def __pickleStuff(filename, stuff):
    save_stuff = open(filename, "wb")
    pickle.dump(stuff, save_stuff)
    save_stuff.close()
def __loadStuff(filename):
    saved_stuff = open(filename,"rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

In [3]:
#Reviews.csv contains amazon fine foods reviews
reviews = pd.read_csv("Reviews.csv")
# Remove null values and unneeded features
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                        'Score','Time'], 1)
reviews = reviews.reset_index(drop=True)
reviews.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
list(reviews.columns.values)

['Summary', 'Text']

In [5]:
type(reviews)

pandas.core.frame.DataFrame

In [6]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [7]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        # We are not using "text.split()" here
        #since it is not fool proof, e.g. words followed by punctuations "Are you kidding?I think you aren't."
        text = re.findall(r"[\w']+", text)
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)# remove links
    text = re.sub(r'\<a href', ' ', text)# remove html link tag
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    return text

In [10]:
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [11]:
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('numberbatch.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 1917248


In [12]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1
                
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Size of Vocabulary: 125885
Number of words missing from CN: 22171
Percent of words that are missing from vocabulary: 17.61%


In [13]:
#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 125885
Number of words we will use: 22701
Percent of words we will use: 18.029999999999998%


In [14]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

22701


In [15]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 49652988
Total number of UNKs in headlines: 338764
Percent of words that are UNK: 0.6799999999999999%


In [16]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
              counts
count  568412.000000
mean        4.181208
std         2.657212
min         0.000000
25%         2.000000
50%         4.000000
75%         5.000000
max        48.000000

Texts:
              counts
count  568412.000000
mean       84.172662
std        83.095176
min         4.000000
25%        35.000000
50%        59.000000
75%       102.000000
max      3530.000000


In [17]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [18]:
max_text_length = 83 # This will cover up to 89.5% lengthes
max_summary_length = 13 # This will cover up to 99% lengthes
min_length = 2
unk_text_limit = 1 # text can contain up to 1 UNK word
unk_summary_limit = 0 # Summary should not contain any UNK word

def filter_condition(item):
    int_summary = item[0]
    int_text = item[1]
    if(len(int_summary) >= min_length and 
       len(int_summary) <= max_summary_length and 
       len(int_text) >= min_length and 
       len(int_text) <= max_text_length and 
       unk_counter(int_summary) <= unk_summary_limit and 
       unk_counter(int_text) <= unk_text_limit):
        return True
    else:
        return False

int_text_summaries = list(zip(int_summaries , int_texts))
int_text_summaries_filtered = list(filter(filter_condition, int_text_summaries))
sorted_int_text_summaries = sorted(int_text_summaries_filtered, key=lambda item: len(item[1]))
sorted_int_text_summaries = list(zip(*sorted_int_text_summaries))
sorted_summaries = list(sorted_int_text_summaries[0])
sorted_texts = list(sorted_int_text_summaries[1])
# Delete those temporary varaibles
del int_text_summaries, sorted_int_text_summaries, int_text_summaries_filtered
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

304812
304812


In [19]:
__pickleStuff("/Users/rishi/Desktop/Reviews/Naive-Approach-Abstract-Summarizer-master/Data/clean_summaries.p",clean_summaries)
__pickleStuff("/Users/rishi/Desktop/Reviews/Naive-Approach-Abstract-Summarizer-master/Data/clean_texts.p",clean_texts)

__pickleStuff("/Users/rishi/Desktop/Reviews/Naive-Approach-Abstract-Summarizer-master/Data/sorted_summaries.p",sorted_summaries)
__pickleStuff("/Users/rishi/Desktop/Reviews/Naive-Approach-Abstract-Summarizer-master/Data/sorted_texts.p",sorted_texts)
__pickleStuff("/Users/rishi/Desktop/Reviews/Naive-Approach-Abstract-Summarizer-master/Data/word_embedding_matrix.p",word_embedding_matrix)

__pickleStuff("/Users/rishi/Desktop/Reviews/Naive-Approach-Abstract-Summarizer-master/Data/vocab_to_int.p",vocab_to_int)
__pickleStuff("/Users/rishi/Desktop/Reviews/Naive-Approach-Abstract-Summarizer-master/Data/int_to_vocab.p",int_to_vocab)

In [20]:
def model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [21]:
def process_encoding_input(target_data, vocab_to_int, batch_size):  
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1]) # slice it to target_data[0:batch_size, 0: -1]
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [22]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    layer_input = rnn_inputs
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    layer_input,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
            layer_input = tf.concat(enc_output, 2)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    return enc_output, enc_state

In [23]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, output_layer,
                            vocab_size, max_summary_length,batch_size):
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,
                                                       helper=training_helper,
                                                       initial_state=dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size),
                                                       output_layer = output_layer)

    training_logits = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [24]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size),
                                                        output_layer)
                
    inference_logits = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [25]:
def lstm_cell(lstm_size, keep_prob):
    cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    return tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob = keep_prob)

def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length,
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    dec_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    output_layer = Dense(vocab_size,kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                     enc_output,
                                                     text_length,
                                                     normalize=False,
                                                     name='BahdanauAttention')
    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,attn_mech,rnn_size)
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input,summary_length,dec_cell,
                                                  output_layer,
                                                  vocab_size,
                                                  max_summary_length,
                                                  batch_size)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,
                                                    vocab_to_int['<GO>'],
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell,
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
    return training_logits, inference_logits

In [26]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size) #shape=(batch_size, senquence length) each seq start with index of<GO>
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    return training_logits, inference_logits

In [27]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [28]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [29]:
print("'<PAD>' has id: {}".format(vocab_to_int['<PAD>']))
sorted_summaries_samples = sorted_summaries[7:50]
sorted_texts_samples = sorted_texts[7:50]
pad_summaries_batch_samples, pad_texts_batch_samples, pad_summaries_lengths_samples, pad_texts_lengths_samples = next(get_batches(
    sorted_summaries_samples, sorted_texts_samples, 5))
print("pad summaries batch samples:\n\r {}".format(pad_summaries_batch_samples))

'<PAD>' has id: 22698
pad summaries batch samples:
 [[  200   225   174   373 22698 22698 22698 22698 22698 22698 22698 22698
  22698]
 [ 3392    89    67   890    67  1410   431   159   317   231 22698 22698
  22698]
 [ 5534  5535   557   558   599   483  2383  1911   672   109    41   330
  22698]
 [  181  1112   557   558   867  3017  1971  2383  1911   672   109    41
    330]
 [ 3392    89    67   890    67  1410   431   159   317   231 22698 22698
  22698]]


In [30]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.95

In [31]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits[0].rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits[0].sample_id, name='predictions')
    
    # Create the weights for sequence_loss, the sould be all True across since each batch is padded
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")
graph_location = "./graph"
print(graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(train_graph)

Instructions for updating:
Use the retry module or similar alternatives.


Instructions for updating:
Use the retry module or similar alternatives.


Graph is built.
./graph


In [32]:
# Subset the data for training
start = 200000
end = start + 10000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 52
The longest text length: 54


In [33]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 20 # If the update loss does not decrease in 20 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "./output/model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch   20/156 - Loss:  4.835, Seconds: 80.40
Epoch   1/100 Batch   40/156 - Loss:  2.948, Seconds: 84.90
Average loss for this update: 3.634
New Record!
Epoch   1/100 Batch   60/156 - Loss:  2.668, Seconds: 79.18
Epoch   1/100 Batch   80/156 - Loss:  2.646, Seconds: 86.62
Epoch   1/100 Batch  100/156 - Loss:  2.572, Seconds: 98.31
Average loss for this update: 2.604
New Record!
Epoch   1/100 Batch  120/156 - Loss:  2.569, Seconds: 92.12
Epoch   1/100 Batch  140/156 - Loss:  2.652, Seconds: 131.03
Average loss for this update: 2.584
New Record!
Epoch   2/100 Batch   20/156 - Loss:  2.580, Seconds: 91.13
Epoch   2/100 Batch   40/156 - Loss:  2.457, Seconds: 89.57
Average loss for this update: 2.461
New Record!
Epoch   2/100 Batch   60/156 - Loss:  2.238, Seconds: 97.56
Epoch   2/100 Batch   80/156 - Loss:  2.191, Seconds: 84.58
Epoch   2/100 Batch  100/156 - Loss:  2.111, Seconds: 82.58
Average loss for this update: 2.153
New Record!
Epoch   2/100 Batch  120/156 - Loss:  2

Epoch  15/100 Batch   80/156 - Loss:  0.434, Seconds: 84.50
Epoch  15/100 Batch  100/156 - Loss:  0.415, Seconds: 103.28
Average loss for this update: 0.431
New Record!
Epoch  15/100 Batch  120/156 - Loss:  0.475, Seconds: 75.60
Epoch  15/100 Batch  140/156 - Loss:  0.499, Seconds: 97.74
Average loss for this update: 0.477
No Improvement.
Epoch  16/100 Batch   20/156 - Loss:  0.493, Seconds: 79.52
Epoch  16/100 Batch   40/156 - Loss:  0.447, Seconds: 76.97
Average loss for this update: 0.469
No Improvement.
Epoch  16/100 Batch   60/156 - Loss:  0.437, Seconds: 81.10
Epoch  16/100 Batch   80/156 - Loss:  0.391, Seconds: 83.27
Epoch  16/100 Batch  100/156 - Loss:  0.371, Seconds: 99.89
Average loss for this update: 0.383
New Record!
Epoch  16/100 Batch  120/156 - Loss:  0.432, Seconds: 92.43
Epoch  16/100 Batch  140/156 - Loss:  0.441, Seconds: 95.09
Average loss for this update: 0.433
No Improvement.
Epoch  17/100 Batch   20/156 - Loss:  0.442, Seconds: 80.83
Epoch  17/100 Batch   40/15

Epoch  29/100 Batch  140/156 - Loss:  0.216, Seconds: 89.98
Average loss for this update: 0.206
No Improvement.
Epoch  30/100 Batch   20/156 - Loss:  0.223, Seconds: 78.04
Epoch  30/100 Batch   40/156 - Loss:  0.177, Seconds: 75.68
Average loss for this update: 0.199
No Improvement.
Epoch  30/100 Batch   60/156 - Loss:  0.191, Seconds: 93.99
Epoch  30/100 Batch   80/156 - Loss:  0.166, Seconds: 84.01
Epoch  30/100 Batch  100/156 - Loss:  0.169, Seconds: 86.98
Average loss for this update: 0.17
New Record!
Epoch  30/100 Batch  120/156 - Loss:  0.196, Seconds: 76.45
Epoch  30/100 Batch  140/156 - Loss:  0.187, Seconds: 93.09
Average loss for this update: 0.19
No Improvement.
Epoch  31/100 Batch   20/156 - Loss:  0.213, Seconds: 86.53
Epoch  31/100 Batch   40/156 - Loss:  0.174, Seconds: 76.07
Average loss for this update: 0.19
No Improvement.
Epoch  31/100 Batch   60/156 - Loss:  0.175, Seconds: 78.86
Epoch  31/100 Batch   80/156 - Loss:  0.167, Seconds: 83.47
Epoch  31/100 Batch  100/15

In [33]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [41]:
fout = open("output.txt","w")

In [36]:
# Create your own review or use one from the dataset
#input_sentence = ""
#text = text_to_seq(input_sentence)

random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[388]
text = text_to_seq(clean_texts[388])

checkpoint = "./output/model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', input_sentence)
print('\nSummary')
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: these nuts arrived fresh and are very tasty they are not broken or dirty looking like cashews sometimes can

Summary
  Response Words: delicious and eh


In [39]:
input_sentence

'these nuts arrived fresh and are very tasty they are not broken or dirty looking like cashews sometimes can'

In [37]:
len(clean_texts)

568412

In [44]:
# Create your own review or use one from the dataset
#input_sentence = ""
#text = text_to_seq(input_sentence)
for n in clean_texts:
    random = np.random.randint(0,len(clean_texts))
    input_sentence = n        #clean_texts[n]
    text = text_to_seq(n)     #(clean_texts[388])

    checkpoint = "./output/model.ckpt"

    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        # Load saved model
        loader = tf.train.import_meta_graph(checkpoint + '.meta')
        loader.restore(sess, checkpoint)

        input_data = loaded_graph.get_tensor_by_name('input:0')
        logits = loaded_graph.get_tensor_by_name('predictions:0')
        text_length = loaded_graph.get_tensor_by_name('text_length:0')
        summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
        keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
        #Multiply by batch_size to match the model's input parameters
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [np.random.randint(5,8)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 

    # Remove the padding from the tweet
    pad = vocab_to_int["<PAD>"] 

    print('Original Text:', input_sentence)
    fout.write('Original Text: '+ input_sentence)
    print('\nSummary')
    print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))
    fout.write('\nSummary')
    fout.write('Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))
    fout.write('\n')

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most

Summary
  Response Words: great quality dog food
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: product arrived labeled as jumbo salted peanuts the peanuts were actually small sized unsalted not sure if this was an error or if the vendor intended to represent the product as jumbo

Summary
  Response Words: poorly sealed
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: this is a confection that has been around a few centuries it is a light pillowy citrus gelatin with nuts in this case filberts and it is cut into tiny squares and then liberally coated with powdered sugar and it is a tiny mouthful of

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: i bought these for my husband who is currently overseas he loves these and apparently his staff likes them also br there are generous amounts of twizzlers in each 16 ounce bag and this was well worth the price a href http www amazon com gp product b001gvisjm twizzlers strawberry 16 ounce bags pack of 6 a

Summary
  Response Words: hubby loves these especially on
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: i can remember buying this candy as a kid and the quality has not dropped in all these years still a superb product you will not be disappointed with

Summary
  Response Words: not fresh
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: i love this candy after weight watchers i had to cut back but still have a craving for it

Summary
  Response Words: do not go by directions
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text:

Original Text: i was visiting my friend nate the other morning for coffee he came out of his storage room with a packet of mccanns instant irish oatmeal he suggested that i try it for my own use in my stash sometimes nate dose not give you a chance to say no so i ended up trying the apple and cinn found it to be very tastefull when made with water or powdered milk it goes good with o j and coffee and a slice of toast and your ready to take on the world or the day at least jerry reith

Summary
  Response Words: i did not get it was
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: i ordered this for my wife as it was reccomended by our daughter she has this almost every morning and likes all flavors she is happy i am happy br a href http www amazon com gp product b001eo5qw8 mccann s instant irish oatmeal variety pack of regular apples cinnamon and maple brown sugar 10 count boxes pack of 6 a

Summary
  Response Words: i love this bread
INFO:tensorflow:Restorin

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: we roast these in a large cast iron pan on the grill about 1 3 of the bag at a time the smell is wonderful and the roasted beans taste delicious too more importantly the coffee is smooth no bitter aftertaste on numerous occasions we have had to send the roasted beans home with friends because they like it so much

Summary
  Response Words: great way to get gushers
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: deal was awesome arrived before halloween as indicated and was enough to satisfy trick or treaters i love the quality of this product and it was much less expensive than the local store s candy

Summary
  Response Words: our new favorite
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: it is chocolate what can i say great variety of everything our family loves with a family of six it goes fast here perfect variety kit kat reeses take five and more

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: it is okay i would not go out of my way to buy it again

Summary
  Response Words: best coconut oil ever
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: no tea flavor at all just whole brunch of artifial flavors it is not returnable i wasted 20 bucks

Summary
  Response Words: one arrived with one star with
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: these looked like a perfect snack to through in with my trail mix unfortunately they arrived in a solid mass of melted chocolate i left them in my pantry for a few days and when i opened them at room temperature they were still gooey i through them in the fridge and i have been breaking off hunks ever since they taste good to me but the chocolate is all grainy after melting and then solidifying again i will not order them online again but if i see them in a store i would pick them up

Summary
  Response

...

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: this hemp oil is excellent stuff for real if you are using fish oil this actually has more omega 3 s and has many other medicinal values br combine this with hemp seeds in smoothies and you will notice a difference in any aches in your knees and your hair will shine and your body will be filled with antioxidants br anything hemp rocks you will not be disappointed in this product and it came on time and was packaged well 3 bottles for this price is a great value

Summary
  Response Words: excellent fish breath
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: the capsules are easy for my wife to take they really help her crohn s disease after reviewing and reasearching we found that hemp seed oil helps crohn s disease she has tried all the prescription meds which if you read the after affects on them they are worse than the crohn s disease this product has really helped her from havin

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: have had this gum from other sources and always liked it this particular shipment seems to have much less peppermint flavor than before could be it is older gum there is no expiration date on it or they are making it with less peppermint now otherwise i received this order very quickly

Summary
  Response Words: xylichew peppermint gum
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: this is an excellent sugar free gum for those who do not want saccarine or aspertame many commercial gums that are advertised as containing xylitol also contain unnatural chemical sweeteners the flavor and soft texture lasts very well in the peppermint and fruit flavors another family member really enjoys the licorice one the chocolate has a disappointing flavor and the cinnamon which to me tastes like almond at first quickly gets hard and loses flavor have not tried spearmint while the serving size is 

INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: i spent some time searching for an alcohol free rum flavor to make daiquiri sorbet and lorann has produced the best flavor thus far which tastes outstanding it blows away the competitors that i have tried baking with it has not yet been attempted but i imagine there would be similar results the price is quite reasonable for a 4 oz bottle although it is much more so if multiple bottles are ordered save on shipping it has natural and artificial ingredients i would prefer it to be all natural but that is a price i am willing to pay for the flavor i predict this will remain my alcohol free rum flavor of choice

Summary
  Response Words: great taste
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: i bought this bag and found it 50 cheaper at smart and final it was also old and and white this was terrible i bought it to put in the candy canisters i give to co workers so i did not find out

Original Text: i did not expect this hot chocolate to be quite as sweet as i found it to be it also has more calories than previous ghiradelli similar hot chocolate br it has a rich taste and if i were not concerned with weight i would overlook the higher calories

Summary
  Response Words: powdered milk
INFO:tensorflow:Restoring parameters from ./output/model.ckpt
Original Text: while you would expect ghiradelli to make good chocolate the flavor of this mocha cocoa is precisely that of great darker chocolate flavor using just enough coffee flavor to accent the cocoa without overtaking it a smooth delicious start or finish to your day even a warm afternoon pick me up on those chilly days

Summary
  Response Words: it is the worst chocolate ever


KeyboardInterrupt: 

In [106]:
ffout = open("output_final.txt",'w')

In [107]:
f = open('output.txt','r')

In [108]:
lines = f.readlines()

In [109]:
lines[1]

'Summary  Response Words: delicious and eh\n'

In [110]:
cnt = 0

In [111]:
for line in lines:
    ffout.write(line)
    ffout.write('\n')